# 評判分析で文章のポジネガを判別しよう

機械学習を用いた評判分析における記念碑的論文( http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf )と同様のセットアップで分析を行い、論文の精度を上回れるかチャレンジしてみましょう！

## 0.前準備

python versionの確認します。<br>
jupyter notebookではシェルコマンドの文頭に"!"をつけるとそのシェルコマンドをnotebook上で実行することができます。<br> 

In [1]:
!python --version

Python 3.6.0


カレントディレクトリの確認とデータディレクトリの確認をします。<br>
osモジュールを使うことでOS依存の機能を使えるようになります。

In [2]:
import os

In [3]:
print( os.listdir(os.path.normpath("./")) )

['評判分析入門_normal.ipynb', 'README.md', '.ipynb_checkpoints', '評判分析入門_advanced.ipynb']


In [4]:
print( os.listdir(os.path.normpath("../dataset/")) )

['README', 'mix20_rand700_tokens.zip', 'tokens']


## 1.dataの読み込みとモジュールのインポート

pyenvなどを用いているとpandasなどがimportできない場合があります。<br>
その可能性の１つとしてlocale（国毎に異なる単位）の設定不足があり得るので、ここではそれを明示的に操作します。<br>

In [5]:
def set_locale():
    default = os.environ.get('LC_ALL')
    print( "Your default locale is", default )
    if default is None:
        os.environ.setdefault('LC_ALL', 'ja_JP.UTF-8')
        print( "Your locale is set as ja_JP.UTF-8" )

set_locale()

Your default locale is None
Your locale is set as ja_JP.UTF-8


今回使うデータファイルのパスをpythonのリストとして取得します。<br>
globはパス名を見つけたりparseしたりするモジュールです( http://docs.python.jp/3/library/glob.html )。<br>
今回扱うデータは https://www.cs.cornell.edu/people/pabo/movie-review-data/ より取得しています。<br>
データ構造は下記のようになっています。<br>
- data
    - README
    - tokens
        - neg
            - file1.txt
            - file2.txt
            - ...
        - pos
            - file1.txt
            - file2.txt
            - ...

In [6]:
import glob

neg_files = glob.glob( os.path.normpath("../dataset/tokens/neg/*") )
pos_files = glob.glob( os.path.normpath("../dataset/tokens/pos/*") )

取得したファイルパスの確認。

In [7]:
print(neg_files[0:2])
print(pos_files[0:2])

['../dataset/tokens/neg/cv424_tok-29318.txt', '../dataset/tokens/neg/cv631_tok-20288.txt']
['../dataset/tokens/pos/cv439_tok-13632.txt', '../dataset/tokens/pos/cv180_tok-20034.txt']


データ読み込みのテストをします。

実際に文章を１つ読み込んでみて正しく読み込めているかを確認します。<br>
本データは１つのファイルに１文で映画のレビュー文章が記載されています。<br>
テキストの読み込みはエンコーディングの問題などでエラーが生じやすいので、慣れるまでは根気強くdebugしましょう。<br>

無事に読み込めたら、具体的に１つファイルの中身を読み込んで内容を確認してみましょう。<br>
sys はファイルサイズ取得などのシステム上の操作を行うモジュールです( http://docs.python.jp/3/library/sys.html )。

In [8]:
import sys

def text_reader(file_path):
    python_version = sys.version_info.major
    
    if python_version >= 3:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                print(line)
    else:
        with open(file_path, 'r') as f:
            for line in f:
                print(line)

In [9]:
text_reader(neg_files[11])

literally blink , and you'll miss _soldier_'s lone flash of wit : a fleeting glimpse of a computer screen listing futuristic supersoldier todd's ( kurt russell ) numerous war commendations , among them the " plissken patch " --referring , of course , to russell's character in john carpenter's _escape_from . . . _ movies . the rest of this sci-fi actioner is brainless junk though writer david webb peoples and director paul anderson serve up an interesting basic premise . in the future , a select few males are chosen at birth to be trained their whole life as soldiers , nothing more ; veteran soldier todd is among the best , if not _the_ best . but when a new genetically engineered brand of soldier is developed , todd and his ilk are rendered obsolete . >from here , the story takes a most uninspired turn . presumed dead , todd is dumped onto a trash dump planet , where he meets up with a peaceful community of people who look and act like extras from _the_postman_ . for reasons that are n

今回使うモジュールの情報をまとめておきます。<br>
詳細な中身などに関してはご自身で調べてみてください。<br>
- matplotlib : グラフなどを描写する<br>
http://matplotlib.org/
- pandas : dataframeでデータを扱い、集計や統計量算出などがすぐ実行できる<br>
http://pandas.pydata.org/
- collections : pythonで扱えるデータの型を提供する<br>
http://docs.python.jp/3/library/collections.html
- numpy : 行列などの数学的オブジェクトを扱う<br>
http://www.numpy.org/
- sklearn.feature_extraction.DictVectorizer : 辞書データをベクトルの形に変換する<br>
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html
- sklearnのモデル : SVM, NB, RF<br>
http://scikit-learn.org/stable/tutorial/basic/tutorial.html
- sklearn.grid_search : パラメタの最適な組み合わせ見つける<br>
http://scikit-learn.org/stable/modules/grid_search.html

In [10]:
import matplotlib # not used in this notebook
import pandas as pd # not used in this notebook

import collections
import numpy as np

from sklearn.feature_extraction import DictVectorizer

from sklearn import svm, naive_bayes
from sklearn.ensemble import RandomForestClassifier

from sklearn import grid_search

/Users/takahi-i/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/takahi-i/.pyenv/versions/3.6.0/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## 2.特徴ベクトルの作成

unigramを作成する関数を定義します。<br>
スペース区切りで単語を抽出し、その数をカウントする単純な関数となります。<br>

In [11]:
def word_counter(string):
    words = string.strip().split()
    count_dict = collections.Counter(words)
    return dict(count_dict)

def get_unigram(file_path):
    result = []
    python_version = sys.version_info.major
    
    if python_version >= 3:
        for file in file_path:
            with open(file, 'r', encoding='utf-8') as f:
                for line in f:
                    count_dict = word_counter(line)
                    result.append(count_dict)
    else:
        for file in file_path:
            with open(file, 'r') as f:
                for line in f:
                    count_dict = word_counter(line)
                    result.append(count_dict)
    
    return result

関数の挙動を確認してみましょう。

In [12]:
word_counter("I am YK. I love data analysis using python.")

{'I': 2,
 'YK.': 1,
 'am': 1,
 'analysis': 1,
 'data': 1,
 'love': 1,
 'python.': 1,
 'using': 1}

この関数を用いて、negative と positive 両方で unigram を作成します。<br>
得られた2つのリストを合わせてモデルのインプット（説明変数）とします。
リストの結合は "+" で実施できます。 ex.) [1] + [2] = [1,2]<br>
negative と positive は各700文ずつありますが、そのうちいくつを使うかをここで指定します。初期設定では全てのデータを使うことになっていますが、後の過程で memory 不足になるようでしたらこの数を減らしてください。<br>

また、 jupyter notebook では %% をつけることで magic commands ( https://ipython.org/ipython-doc/3/interactive/magics.html ) という便利なコマンドを実行できます。ここでは処理にかかる時間をセルに表示するコマンドを使用しています。

In [13]:
%%time

DATA_NUM = 700

unigrams_data = get_unigram(neg_files[:DATA_NUM]) + get_unigram(pos_files[:DATA_NUM])

CPU times: user 305 ms, sys: 135 ms, total: 441 ms
Wall time: 616 ms


得られたunigram_dataを確認してみます。単語の出現数がカウントされていることが確認できます。<br>
合わせてそのデータサイズも確認してみます。

In [14]:
print( unigrams_data[0] )
print( "data size :", sys.getsizeof(unigrams_data) / 1000000, "[MB]" )

{'summer': 5, 'catch': 5, '(': 3, '2001': 1, ')': 3, '1': 1, '1/2': 1, 'stars': 1, 'out': 2, 'of': 14, '4': 1, '.': 48, 'starring': 1, 'freddie': 2, 'prinze': 3, 'jr': 2, ',': 30, 'jessical': 1, 'biel': 3, 'matthew': 2, 'lillard': 1, 'fred': 1, 'ward': 1, 'jason': 1, 'gedrick': 1, 'brittany': 1, 'murphy': 1, 'bruce': 2, 'davison': 2, 'and': 12, 'brian': 1, 'dennehy': 1, 'screenplay': 2, 'by': 8, 'kevin': 2, 'falls': 5, 'john': 2, 'gatins': 2, 'story': 4, 'directed': 1, 'mike': 1, 'tollin': 1, 'rated': 1, 'pg-13': 1, 'approx': 1, '110': 1, 'minutes': 1, 'is': 12, 'a': 14, 'minor': 1, 'league': 3, 'effort': 1, 'nine': 1, 'innings': 1, 'banality': 1, 'with': 2, 'lineup': 1, 'stock': 1, 'situations': 2, 'stereotypical': 1, 'characters': 1, 'this': 5, 'feature': 1, 'like': 1, 'double': 1, 'header': 1, 'two': 1, 'sets': 1, 'clich駸': 1, 'for': 5, 'the': 22, 'price': 1, 'one': 2, 'not': 1, 'only': 3, 'do': 1, 'we': 2, 'get': 1, 'usual': 1, 'tired': 1, 'sports': 1, 'chestnuts': 1, 'but': 3, 'ba

上で得られたデータは unigram という 1400 の要素を持つリストであり、各要素は key と value からなる辞書となっています。<br>
これを扱いやすい行列の形にします。<br>
ここでは各行が１つのレビューテキストに対応するようにして、各列が単語、要素がその単語の出現数というデータを作成します。<br>
scikit-learn で実装されている DictVectorizer という関数を使うことでそれが簡単に実行できます。<br>

In [15]:
%%time
vec = DictVectorizer()
feature_vectors_csr = vec.fit_transform( unigrams_data )

CPU times: user 497 ms, sys: 22.9 ms, total: 520 ms
Wall time: 527 ms


作成したデータを確認してみます。

In [16]:
feature_vectors_csr

<1400x44219 sparse matrix of type '<class 'numpy.float64'>'
	with 496525 stored elements in Compressed Sparse Row format>

行列の全成分（行成分×列成分）は 60,000,000 要素くらいありますが、このうちのほとんどは 0 で 0 以外の値が入っているのは500,000程度です。<br>
この CSR(Compressed Sparse Row) matrix というのはこのような疎行列をの 0 でない成分だけを保持する賢いものになっています。<br>

一方で 0 の成分を陽に保って普通の行列としてデータを保持することも可能です。<br>

In [17]:
feature_vectors = vec.fit_transform( unigrams_data ).toarray()
print( "data dimension :", feature_vectors.shape )
print( feature_vectors[0] )
print( "data size :", sys.getsizeof(feature_vectors) / 1000000, "[MB]" )

data dimension : (1400, 44219)
[0. 0. 6. ... 0. 0. 0.]
data size : 495.252912 [MB]


こちらはデータが非常に大きくなっていますが、これは 0 という成分を陽に保持しているためです。<br>
この段階で memory error が生じる場合は一度 kernel を restart して DATA_NUM の数を減らして再実行してください。<br>

## 3.ラベルデータの作成

今回扱うデータセットは全てに negative, positive というラベルが振られています。<br>
ここではそのラベルを neagtive → 0, neagtive → 1 とすることで二値判別問題のセットアップを構築します。<br>
先ほど作った説明変数となる特徴ベクトルはnegative sample 700文とpositive sample 700文を縦につなげて作ったものなので、0が700個と1が700個並んでいるベクトルを作成すれば必要なラベルを作れます。<br>

In [18]:
labels = np.r_[np.tile(0, DATA_NUM), np.tile(1, DATA_NUM)]

正しい位置で0と1の振替がなされているか確認します。

In [19]:
print( labels[0], labels[DATA_NUM-1], labels[DATA_NUM], labels[2*DATA_NUM-1]  )

0 0 1 1


## 4.学習用データとテスト用データの作成方法

論文の記述によれば、データを偏りがないように3分割に分け、 three fold cross validation でモデルを評価しています。<br>
ここでは乱数を生成して、データを3等分することで同様の状況を再現することにします。<br>
結果の再現性を担保するために乱数の seed も設定しておきます。<br>

In [20]:
np.random.seed(7789)

shuffle_order = np.random.choice( 2*DATA_NUM, 2*DATA_NUM, replace=False )

生成した乱数の中身を確認します。

In [21]:
print( "length :", len(shuffle_order) )
print( "first 10 elements :", shuffle_order[0:10] )

length : 1400
first 10 elements : [1235 1232  910  162  343 1160  221  545 1112 1322]


分割したデータセットに含まれるラベル=1の数を数えることでデータの偏りが生じていないかを確認します。<br>
明らかに偏りが生じてしまった場合は乱数のseedを設定し直します。<br>

In [22]:
one_third_size = int( 2*DATA_NUM / 3. )
print( "one third of the length :", one_third_size )

print( "# of '1' in 1st set :", np.sum( labels[ shuffle_order[:one_third_size] ]  ) )
print( "# of '1' in 2nd set :", np.sum( labels[ shuffle_order[one_third_size:2*one_third_size] ]  ) )
print( "# of '1' in 3rd set :", np.sum( labels[ shuffle_order[2*one_third_size:] ]  ) )

one third of the length : 466
# of '1' in 1st set : 227
# of '1' in 2nd set : 233
# of '1' in 3rd set : 240


## 5.モデルを学習して精度を検証

学習に必要な関数を定義します。<br>
ここではモデルとして{Support Vector Machine(SVM), Naive Bayes(NB), Random Forest(RF)}を用います。<br>
モデルの性能測定は予測と答えが一致する数をカウントして正答率を求めることで実施します。<br>

与えられたリストをN分割する関数を定義します。<br>
割り切れない場合はうしろのリストに格納します。<br>

In [23]:
def N_splitter(seq, N):
    avg = len(seq) / float(N)
    out = []
    last = 0.0
    
    while last < len(seq):
        out.append( seq[int(last):int(last + avg)] )
        last += avg
        
    return np.array(out)

望ましい動作をするか確認してみます。

In [24]:
N_splitter(range(14), 3)

array([range(0, 4), range(4, 9), range(9, 14)], dtype=object)

モデルの学習や予測のための関数を定義します。<br>
- train_model : 説明変数とラベルと手法を与えることでモデルを学習する
- predict : モデルと説明変数を与えることでラベルを予測する
- evaluate_model : 予測したラベルと実際の答えの合致数を調べる
- cross_validate : cross_validationを実行する

In [25]:
def train_model(features, labels, method='SVM', parameters=None):
    ### set the model
    if method == 'SVM':
        model = svm.SVC()
    elif method == 'NB':
        model = naive_bayes.GaussianNB()
    elif method == 'RF':
        model = RandomForestClassifier()
    else:
        print("Set method as SVM (for Support vector machine), NB (for Naive Bayes) or RF (Random Forest)")
    ### set parameters if exists
    if parameters:
        model.set_params(**parameters)
    ### train the model
    model.fit( features, labels )
    ### return the trained model
    return model

def predict(model, features):
    predictions = model.predict( features )
    return predictions

def evaluate_model(predictions, labels):
    data_num = len(labels)
    correct_num = np.sum( predictions == labels )
    return data_num, correct_num

def cross_validate(n_folds, feature_vectors, labels, shuffle_order, method='SVM', parameters=None):
    result_test_num = []
    result_correct_num = []
    
    n_splits = N_splitter( range(2*DATA_NUM), n_folds )

    for i in range(n_folds):
        print( "Executing {0}th set...".format(i+1) )
        
        test_elems = shuffle_order[ n_splits[i] ]
        train_elems = np.array([])
        train_set = n_splits[ np.arange(n_folds) !=i ]
        for j in train_set:
            train_elems = np.r_[ train_elems, shuffle_order[j] ]
        train_elems = train_elems.astype(np.integer)

        # train
        model = train_model( feature_vectors[train_elems], labels[train_elems], method, parameters )
        # predict
        predictions = predict( model, feature_vectors[test_elems] )
        # evaluate
        test_num, correct_num = evaluate_model( predictions, labels[test_elems] )
        result_test_num.append( test_num )
        result_correct_num.append( correct_num )
    
    return result_test_num, result_correct_num

上記関数はCross validationの数を変数として設定できるように作ってあります。<br>
今回は上述のように 3-folds で分析を行います。<br>

In [26]:
N_FOLDS = 3

準備ができたのでここでモデルを学習してその精度を確認してみましょう。<br>
とりあえず何も考えずに準備した Bag Of Words をインプットにしてモデルを学習し、その精度を確認してみます。<br>

In [27]:
%%time
ans,corr = cross_validate(N_FOLDS, feature_vectors_csr, labels, shuffle_order, method='SVM', parameters=None)

Executing 1th set...
Executing 2th set...
Executing 3th set...
CPU times: user 11.3 s, sys: 76.5 ms, total: 11.4 s
Wall time: 11.4 s


結果を確認します。

In [28]:
print( "average precision : ", np.around( 100.*sum(corr)/sum(ans), decimals=1 ), "%" )

average precision :  62.9 %


最後の一文などで確かに positive な評価をしているレビューだということが見て取れます。

## 6.パラメタチューニング

grid searchでパラメタをチューニングすることで、どの程度精度が上がるかを確認してみます。<br>
scikit-learnにはgrid searchが実装されているので、ここではそれを用いてパラメタをチューニングしてみます。<br>
grid search に関しては http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html など。<br>
下記セルの実行には4,5分程度かかります。

In [29]:
%%time

search_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4], 'C': [0.1, 1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100, 1000]}
]

model = svm.SVC()
clf = grid_search.GridSearchCV(model, search_parameters)
clf.fit( feature_vectors_csr, labels )

CPU times: user 3min 53s, sys: 1.01 s, total: 3min 54s
Wall time: 3min 56s


grid searchによって発見したパラメタやスコアを確認してみます。

In [30]:
print("best paremters : ", clf.best_params_)
print("best scores : ", clf.best_score_)

best paremters :  {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
best scores :  0.7935714285714286
